In [4]:
!pip install sentence-transformers rake_nltk scispacy

     |████████████████████████████████| 79 kB 1.8 MB/s eta 0:00:011
     |████████████████████████████████| 44 kB 1.5 MB/s  eta 0:00:01
     |████████████████████████████████| 4.2 MB 4.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.5 MB 40.1 MB/s eta 0:00:01
     |████████████████████████████████| 13.5 MB 32.0 MB/s eta 0:00:01
     |████████████████████████████████| 6.0 MB 42.4 MB/s eta 0:00:01
     |████████████████████████████████| 71 kB 5.5 MB/s  eta 0:00:01
     |████████████████████████████████| 188 kB 54.3 MB/s eta 0:00:01
     |████████████████████████████████| 181 kB 39.8 MB/s eta 0:00:01
     |████████████████████████████████| 660 kB 42.5 MB/s eta 0:00:01
     |████████████████████████████████| 42 kB 861 kB/s  eta 0:00:01
     |████████████████████████████████| 457 kB 46.9 MB/s eta 0:00:01
     |████████████████████████████████| 86 kB 3.1 MB/s  eta 0:00:01
     |████████████████████████████████| 6.6 MB 39.9 MB/s eta 0:00:01
  Created wheel for sentence-transforme

In [7]:
!python -m spacy validate

2022-06-14 18:30:07.979342: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2022-06-14 18:30:07.979402: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
     |████████████████████████████████| 777.4 MB 3.7 kB/s  eta 0:00:01     |███████▉                        | 190.5 MB 35.4 MB/s eta 0:00:17     |████████████▍                   | 300.2 MB 50.9 MB/s eta 0:00:10     |███████████████████▋            | 476.9 MB 43.9 MB/s eta 0:00:07��█▊            | 478.2 MB 43.9 MB/s eta 0:00:07██████████████▏           | 490.9 MB 43.9 MB/s eta 0:00:07
  Attempting uninstall: en-core-web-lg
    Found existing installation: en-core-web-lg 2.3.1
    Uninstalling en-core-web-lg-2.3.1:
      Successfully uninstalled en-core-web-lg-2.3.1
✔ Downlo

In [8]:
import os
import gc
import time
import sys
import re
import random
import json
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
import math
import shutil
import string
import pickle
import joblib
import asyncio
import scipy as sp
import networkx as nx
from joblib import Parallel, delayed
from pathlib import Path
import dask.dataframe as dd
from dask.diagnostics import ProgressBar

import cv2
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from sklearn.preprocessing import MinMaxScaler

import torch
import transformers
import tokenizers
print(f"torch.__version__: {torch.__version__}")
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from tqdm import tqdm
from torch.optim import Adam, SGD, AdamW
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig, AutoModel, get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from torch.utils.data import DataLoader, Dataset
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint

import itertools
import unicodedata
import seaborn as sns
import nltk
from nltk.corpus import stopwords, wordnet
from itertools import groupby
from sentence_transformers import SentenceTransformer
import spacy
from rake_nltk import Rake
from spacy.tokenizer import Tokenizer
from spacy.matcher import PhraseMatcher
from spacy.lang.char_classes import CONCAT_QUOTES, LIST_ELLIPSES, LIST_ICONS, ALPHA, ALPHA_LOWER, ALPHA_UPPER
from spacy.util import compile_infix_regex
from scispacy.abbreviation import AbbreviationDetector
from spacy.pipeline import EntityRecognizer

nlp = spacy.load('en_core_web_lg')
#nlp.add_pipe("abbreviation_detector")

re_token_match = spacy.tokenizer._get_regex_pattern(nlp.Defaults.token_match)

torch.__version__: 1.7.1+cpu
tokenizers.__version__: 0.10.3
transformers.__version__: 4.5.1


In [9]:
pd.set_option('display.precision', 4)

cm = sns.light_palette('green', as_cmap=True)
props_param = "color:white; font-weight:bold; background-color:green;"

CUSTOM_SEED = 42
CUSTOM_BATCH = 24
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Stopwords and infixes
ADDITIONAL_STOPWORDS = ['one or more', 'a', 'needn', 'a', 'not', 'able', 'never', 'about', 'needn’t', 'accordance', 'now', 'often', 'above', 'no', 'according', 'of', 'mentioned', 'others', 'after', 'nor', 'all', 'on', 'accordingly', 'otherwise', 'again', 'not', 'also', 'onto', 'across', 'overall', 'against', 'now', 'an', 'or', 'along', 'rather', 'ain', 'o', 'and', 'other', 'already', 'remarkably', 'all', 'of', 'another', 'particularly', 'alternatively', 'significantly', 'am', 'off', 'are', 'preferably', 'always', 'simply', 'an', 'on', 'as', 'preferred', 'among', 'sometimes', 'and', 'once', 'at', 'present', 'and/or', 'specifically', 'any', 'only', 'be', 'provide', 'anything', 'straight', 'are', 'or', 'because', 'provided', 'anywhere', 'forward', 'aren', 'other', 'been', 'provides', 'better', 'substantially', 'aren’t', 'our', 'being', 'relatively', 'disclosure', 'thereafter', 'as', 'ours', 'by', 'respectively', 'due', 'therebetween', 'at', 'ourselves', 'claim', 'said', 'easily', 'therefor', 'be', 'out', 'comprises', 'comprising', 'should', 'easy', 'therefrom', 'because', 'over', 'since', 'e.g', 'therein', 'been', 'own', 'could', 'some', 'either', 'thereinto', 'before', 're', 'described', 'such', 'elsewhere', 'thereon', 'being', 's', 'desired', 'suitable', 'enough', 'therethrough', 'below', 'same', 'do', 'than', 'especially', 'therewith', 'between', 'shan', 'does', 'that', 'essentially', 'together', 'both', 'shan’t', 'each', 'the', 'et', 'al', 'toward', 'but', 'she', 'embodiment', 'their', 'etc', 'towards', 'by', 'she’s', 'fig', 'then', 'eventually', 'typical', 'can', 'should', 'figs', 'there', 'excellent', 'upon', 'couldn', 'should’ve', 'for', 'thereby', 'finally', 'via', 'couldn’t', 'shouldn', 'from', 'therefore', 'furthermore', 'vice', 'versa', 'd', 'shouldn’t', 'further', 'thereof', 'good', 'whatever', 'did', 'so', 'generally', 'thereto', 'hence', 'whereas', 'didn', 'some', 'had', 'these', 'he/she', 'whereat', 'didn’t', 'such', 'has', 'they', 'him/her', 'wherever', 'do', 't', 'have', 'this', 'his/her', 'whether', 'does', 'than', 'having', 'those', 'ie', 'whose', 'doesn', 'that', 'herein', 'thus', 'ii', 'within', 'doesn’t', 'that’ll', 'however', 'to', 'iii', 'without', 'doing', 'the', 'if', 'use', 'instead', 'yet', 'don', 'their', 'in', 'various', 'later', 'don’t', 'theirs', 'into', 'was', 'like', 'down', 'them', 'invention', 'were', 'little', 'during', 'themselves', 'is', 'what', 'many', 'each', 'there', 'it', 'when', 'may', 'few', 'these', 'its', 'where', 'meanwhile', 'for', 'they', 'means', 'whereby', 'might', 'from', 'this', 'wherein', 'moreover', 'further', 'those', 'which', 'much', 'had', 'through', 'while', 'must', 'hadn', 'to', 'who', 'hadn’t', 'too', 'will', 'has', 'under', 'with', 'hasn', 'until', 'Would', 'hasn’t', 'up', 'have', 've', 'haven', 'very', 'haven’t', 'was', 'having', 'wasn', 'he', 'wasn’t', 'her', 'we', 'here', 'were', 'hers', 'weren', 'herself', 'weren’t', 'him', 'what', 'himself', 'when', 'his', 'where', 'how', 'which', 'i', 'while', 'if', 'who', 'in', 'whom', 'into', 'why', 'is', 'will', 'isn', 'with', 'isn’t', 'won', 'it', 'won’t', 'it’s', 'wouldn', 'its', 'wouldn’t', 'itself', 'y', 'just', 'you', 'll', 'you’d', 'm', 'you’ll', 'ma', 'you’re', 'me', 'you’ve', 'mightn', 'your', 'mightn’t', 'yours', 'more', 'yourself', 'most', 'yourselves', 'mustn', 'mustn’t', 'my', 'myself']

puncts = ['\u200d','?', '....','..','...','', ',', '.', '"', ':', ')', '(', '-', '!', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '*', '+', '\\',
    '•', '~', '£', '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█',
    '½', 'à', '…', '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓',
    '—', '‹', '─', '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾',
    'Ã', '⋅', '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø',
    '¹', '≤', '‡', '√', '!','🅰','🅱']

infixes = (
    LIST_ELLIPSES
    + LIST_ICONS
    + [
        r"(?<=[0-9])[+\-\*^](?=[0-9-])",
        r"(?<=[{al}{q}])\.(?=[{au}{q}])".format(
            al=ALPHA_LOWER, au=ALPHA_UPPER, q=CONCAT_QUOTES
        ),
        r"(?<=[{a}]),(?=[{a}])".format(a=ALPHA),
        # ✅ Commented out regex that splits on hyphens between letters:
        # r"(?<=[{a}])(?:{h})(?=[{a}])".format(a=ALPHA, h=HYPHENS),
        r"(?<=[{a}0-9])[:<>=/](?=[{a}])".format(a=ALPHA),
        r'''[-~]'''
    ]
)

table = {
'A': 'Human Necessities',
'B': 'Operations and Transport',
'C': 'Chemistry and Metallurgy',
'D': 'Textiles',
'E': 'Fixed Constructions',
'F': 'Mechanical Engineering',
'G': 'Physics',
'H': 'Electricity',
'Y': 'Emerging Cross-Sectional Technologies'
}

In [10]:
OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [11]:
def get_logger(filename=OUTPUT_DIR+'train'):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=f"{filename}.log")
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    return logger

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def get_score(y_true, y_pred):
    score = sp.stats.pearsonr(y_true, y_pred)[0]
    return score

def inference_fn(test_loader, model, device, is_sigmoid=True):
    preds = []
    model.eval()
    model.to(device)
    tk0 = tqdm(test_loader, total=len(test_loader))

    for inputs in tk0:
        for k, v in inputs.items():
            inputs[k] = v.to(device)

        with torch.no_grad():
            output = model(inputs)

        if is_sigmoid == True:
            preds.append(output.sigmoid().to('cpu').numpy())
        else:
            preds.append(output.to('cpu').numpy())

    return np.concatenate(preds)


def upd_outputs(data, is_trim=False, is_minmax=False, is_reshape=False):
    min_max_scaler = MinMaxScaler()

    if is_trim == True:
        data = np.where(data <=0, 0, data)
        data = np.where(data >=1, 1, data)

    if is_minmax ==True:
        data = min_max_scaler.fit_transform(data)

    if is_reshape == True:
        data = data.reshape(-1)

    return data


def parse_sections(out):
    title = out.split(patent)[1].split(' - ')[1].split('\n')[0].strip()
    out = out.split(title)[3].split('Abstract')[1].split(' US1')[0]
    out = out.replace('.\n', '. ').replace('\n', ' ').replace('\r', '')
    out = 'ABSTRACT ' + ' '.join(out.split()).strip()

    abstract = out.split('ABSTRACT ')[1].split('BACKGROUND')[0].strip()

    if 'BRIEF DESCRIPTION' in out:
        background = out.split('BACKGROUND ')[1].split('BRIEF DESCRIPTION')[0].strip()
        brief_description = out.split('BRIEF DESCRIPTION ')[1].split('DESCRIPTION OF DRAWINGS')[0].strip()
    else:
        background = out.split('BACKGROUND ')[1].split('SUMMARY ')[0].strip()
        brief_description = out.split('SUMMARY ')[1].split('DESCRIPTION OF DRAWINGS')[0].strip()

    if 'DESCRIPTION OF EMBODIMENTS' in out:
        desc = 'DESCRIPTION OF EMBODIMENTS '
    else:
        desc = 'DETAILED DESCRIPTION '

    if 'The invention claimed is:' in out:
        detailed_description = out.split(desc)[1].split('The invention claimed is:')[0].strip()
        claims = out.split('The invention claimed is: ')[1].split(' US1')[0]
    elif 'What is claimed is:' in out:
        detailed_description = out.split(desc)[1].split('What is claimed is:')[0].strip()
        claims = out.split('What is claimed is: ')[1].split(' US1')[0]
    else:
        detailed_description = out.split(desc)[1].split('Claims')[0].strip()
        claims = out.split('Claims')[1].split(' US1')[0]

    return abstract, background, brief_description, detailed_description, claims

def flatten(l):
    """
    Flatten list of lists.
    """
    import collections

    for el in l:
        if isinstance(
                el, collections.Iterable) and not isinstance(
                el, (str, bytes)):
            for ell in flatten(el):
                yield ell
        else:
            yield el

def split_claims(claims):
    nums = list(map(str, re.findall(r'\d+\.\s', claims)))
    claims_dict = {}
    total_claims = len(nums)
    for ix, num in enumerate(nums):
         if ix == total_claims - 1:
             claims_dict[f'Claim {num}'.replace('. ', '')] = claims.split(num)[1]
             break
         claims_dict[f'Claim {num}'.replace('. ', '')] = claims.split(num)[1].split(nums[ix+1])[0]
    return claims_dict

def tokenize(sentence, infixes):
    all_stopwords = spacy.lang.en.stop_words.STOP_WORDS
    prefix_re = spacy.util.compile_prefix_regex(nlp.Defaults.prefixes)
    suffix_re = spacy.util.compile_suffix_regex(nlp.Defaults.suffixes)
    infix_re = compile_infix_regex(infixes)

    def customize_tokenizer(nlp):
        # Adds support to use `-` as the delimiter for tokenization
        return Tokenizer(nlp.vocab, prefix_search=prefix_re.search,
                         suffix_search=suffix_re.search,
                         infix_finditer=infix_re.finditer,
                         token_match=None
                        )

    nlp.tokenizer = customize_tokenizer(nlp)
    return [token.text for token in sentence if not token.is_stop]

def remove_from_list(x, stuff_to_remove) -> list:
    for item in stuff_to_remove:
        # Making sure to iterate through the entire token
        for i,token in enumerate(x):
            if item == token:
                del x[i]
    return x

def Remove_Duplicates(text_in):
    return re.sub(r"\b(\w+)(?:\W\1\b)+", r"\1", text_in, flags=re.IGNORECASE)


def remove_consecutive_nums(text):
    # Remove any chunks of consecutive numbers
    number_strings = re.findall(r'\d+[ \t]\d+', text)
    ind_num_strings = []
    for j in number_strings:
        x = [int(i) for i in j.split()]
        ind_num_strings.append(x)

    flat_num_list = [item for sublist in ind_num_strings for item in sublist]

    for i in flat_num_list:
        j=re.sub(r'\d+','',str(i))
        text = text.replace(str(i),j)
    return text


def basic_clean(text_list, infixes, stopwords):
    """
    A simple function to clean up the data. All the words that
    are not designated as a stop word is then lemmatized after
    encoding and basic regex parsing are performed.
    """

    text_list_clean = []
    for text in text_list:
        text = re.sub(r'[\)\(\.\,\;\\\?\&\%\!\+\-]', '', re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\xff\xad\x0c6§\\\£\Â*_<>""⎫•{}Γ~]', ' ', str(' '.join(re.split('\s*-\s*', text)))))
        if len(text.split("  ")) > 1000:
            text = " ".join(["".join(w.split(" ")) if len(w.split(' '))>1 else w for w in text.split("  ")])
        text_list_clean.append([i for i in remove_from_list(re.sub('\s+', ' ', re.sub('\s\s+', ' ', re.sub('\s+\s+', ' ', Remove_Duplicates(re.sub(r"\b(?=[mdclxvii])m{0,4}(cm|cd|d?c{0,3})(xc|xl|l?x{0,3})([ii]x|[ii]v|v?[ii]{0,3})\b\.?", '', (unicodedata.normalize('NFKD', re.sub(' +', ' ', re.sub(r"\s+\s+"," ", re.sub(r"\\,",",", re.sub(r" \,",",", re.sub(r"\\.",".", re.sub(r" \.",".", re.sub(r"\(\s+\)","", re.sub(r"\(\)","", re.sub(r" \)","", re.sub(r"\( ","", remove_consecutive_nums(re.sub(r"\s+"," ", re.sub(r"([A-z])\- ([A-z])", r"\1\2", re.sub(r'\s', ' ', text)).replace('\'','').replace('. .', '.').replace('\'',''))))))))))))).lower())
        .encode('ascii', 'ignore')
        .decode('utf-8', 'ignore')
        .lower())))))).split(), puncts) if not i.isdigit() or i in stopwords])
        del text

    return '. '.join(x.strip().capitalize() for x in '. '.join(' '.join([word for word in sent]) for sent in text_list_clean).split('.')) + '.'


def get_cpc_texts():
    """
    Function taken from Y Nakama's notebook:
    https://www.kaggle.com/code/yasufuminakama/pppm-deberta-v3-large-baseline-w-w-b-train
    """
    contexts = []
    pattern = '[A-Z]\d+'
    for file_name in os.listdir('cpc-data/CPCSchemeXML202105'):
        result = re.findall(pattern, file_name)
        if result:
            contexts.append(result)
    contexts = sorted(set(sum(contexts, [])))
    results = {}
    for cpc in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'Y']:
        with open(f'cpc-data/CPCTitleList202202/cpc-section-{cpc}_20220201.txt') as f:
            s = f.read()
        pattern = f'{cpc}\t\t.+'
        result = re.findall(pattern, s)
        cpc_result = result[0].lstrip(pattern)
        for context in [c for c in contexts if c[0] == cpc]:
            pattern = f'{context}\t\t.+'
            result = re.findall(pattern, s)
            results[context] = cpc_result + ". " + result[0].lstrip(pattern)
    return results

In [ ]:
# train = pd.read_csv(f"{INPUT_DIR}train.csv")
# test = pd.read_csv(f"{INPUT_DIR}test.csv")
# submission = pd.read_csv(f"{INPUT_DIR}sample_submission.csv")
# print(f"train.shape: {train.shape}")
# print(f"test.shape: {test.shape}")
# print(f"submission.shape: {submission.shape}")
# train.head()
# test.head()
# submission.head()

# train['general_context'] = train['context'].apply(lambda x: table[x[0].upper()])
# test['general_context'] = test['context'].apply(lambda x: table[x[0].upper()])

# train = pd.concat([train, pd.get_dummies(train['general_context'])], axis=1)
# test = pd.concat([test, pd.get_dummies(test['general_context'])], axis=1)

# cpc_texts = get_cpc_texts()
# torch.save(cpc_texts, f"{OUTPUT_DIR}cpc_texts.pth")
# train['context_text'] = train['context'].map(cpc_texts)
# test['context_text'] = test['context'].map(cpc_texts)


# train['section'] = train['context'].astype(str).str[0]
# train['classes'] = train['context'].astype(str).str[1:]

# train['num_anchor'] = train['anchor'].str.contains('[0-9]', na=False)
# train['anchor_len'] = train['anchor'].str.split().str.len()
# train['num_target'] = train['target'].str.contains('[0-9]', na=False)
# train['target_len'] = train['target'].str.split().str.len()
# train.target_len.value_counts()

# Config

In [ ]:
seed_everything(CUSTOM_SEED)

LOGGER = get_logger()

class CFG:
    competition='PPPM'
    debug=False
    apex=True
    print_freq=100
    num_workers=4
    #input_path = '../input/traintestcleaned'
    input_path = './'
    model="microsoft/deberta-v3-large"
    scheduler='cosine' # ['linear', 'cosine']
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=0
    epochs=4
    encoder_lr=2e-5
    decoder_lr=2e-5
    min_lr=1e-6
    eps=1e-6
    betas=(0.9, 0.999)
    batch_size=19
    fc_dropout=0.2
    target_size=1
    max_len=512
    weight_decay=0.01
    gradient_accumulation_steps=1
    max_grad_norm=1000
    seed=42
    n_fold=5
    trn_fold=[0, 1, 2, 3, 4]
    train=True

In [13]:
train = pd.read_csv(f"{CFG.input_path}train_clean.csv")

# Tokenizer

In [15]:
def prepare_input(cfg, text):
    inputs = cfg.tokenizer(text,
                           add_special_tokens=True,
                           max_length=cfg.max_len,
                           padding="max_length",
                           return_offsets_mapping=False)
    for k, v in inputs.items():
        inputs[k] = torch.tensor(v, dtype=torch.long)
    return inputs

if CFG.debug:
    CFG.epochs = 2
    CFG.trn_fold = [0]

train['score_map'] = train['score'].map({0.00: 0, 0.25: 1, 0.50: 2, 0.75: 3, 1.00: 4})
Fold = StratifiedKFold(n_splits=CFG.n_fold, shuffle=True, random_state=CFG.seed)
for n, (train_index, val_index) in enumerate(Fold.split(train, train['score_map'])):
    train.loc[val_index, 'fold'] = int(n)
train['fold'] = train['fold'].astype(int)
train.groupby('fold').size()

if CFG.debug:
    train.groupby('fold').size()
    train = train.sample(n=1000, random_state=0).reset_index(drop=True)
    train.groupby('fold').size()

tokenizer = AutoTokenizer.from_pretrained(CFG.model)
tokenizer.save_pretrained(OUTPUT_DIR + 'tokenizer/')
CFG.tokenizer = tokenizer

Downloading:   0%|          | 0.00/580 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


# Token Lengths

In [16]:
#cpc_texts = torch.load('../input/cpc-texts/cpc_texts.pth')
cpc_texts = torch.load('cpc_texts.pth')
lengths_dict = {}

lengths = []
tk0 = tqdm(cpc_texts.values(), total=len(cpc_texts))
for text in tk0:
    length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
    lengths.append(length)
lengths_dict['context_text'] = lengths

for text_col in ['anchor', 'target']:
    lengths = []
    tk0 = tqdm(train[text_col].fillna("").values, total=len(train))
    for text in tk0:
        length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
        lengths.append(length)
    lengths_dict[text_col] = lengths

CFG.max_len = max(lengths_dict['anchor']) + max(lengths_dict['target']) + max(lengths_dict['context_text']) + 4
LOGGER.info(f"max_len: {CFG.max_len}")

100%|██████████| 36473/36473 [00:03<00:00, 10554.05it/s]
max_len: 133


# Dataset

In [17]:
class TrainDataset(Dataset):
    def __init__(self, cfg, df):
        self.cfg = cfg
        self.texts = df['text'].values
        self.labels = df['score'].values

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, item):
        inputs = prepare_input(self.cfg, self.texts[item])
        label = torch.tensor(self.labels[item], dtype=torch.float)
        return inputs, label

In [18]:
class CustomModel(nn.Module):
    def __init__(self, cfg, config_path=None, pretrained=False):
        super().__init__()
        self.cfg = cfg
        if config_path is None:
            self.config = AutoConfig.from_pretrained(cfg.model, output_hidden_states=True)
        else:
            self.config = torch.load(config_path)
        if pretrained:
            self.model = AutoModel.from_pretrained(cfg.model, config=self.config)
        else:
            self.model = AutoModel.from_config(self.config)
        self.fc_dropout = nn.Dropout(cfg.fc_dropout)
        self.fc = nn.Linear(self.config.hidden_size, self.cfg.target_size)
        self._init_weights(self.fc)
        self.attention = nn.Sequential(
            nn.Linear(self.config.hidden_size, 512),
            nn.Tanh(),
            nn.Linear(512, 1),
            nn.Softmax(dim=1)
        )
        self._init_weights(self.attention)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.bias is not None:
                module.bias.data.zero_()
        elif isinstance(module, nn.Embedding):
            module.weight.data.normal_(mean=0.0, std=self.config.initializer_range)
            if module.padding_idx is not None:
                module.weight.data[module.padding_idx].zero_()
        elif isinstance(module, nn.LayerNorm):
            module.bias.data.zero_()
            module.weight.data.fill_(1.0)

    def feature(self, inputs):
        outputs = self.model(**inputs)
        last_hidden_states = outputs[0]
        # feature = torch.mean(last_hidden_states, 1)
        weights = self.attention(last_hidden_states)
        return torch.sum(weights * last_hidden_states, dim=1)

    def forward(self, inputs):
        feature = self.feature(inputs)
        return self.fc(self.fc_dropout(feature))

# Train

In [19]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


def train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device):
    model.train()
    scaler = torch.cuda.amp.GradScaler(enabled=CFG.apex)
    losses = AverageMeter()
    start = end = time.time()
    global_step = 0
    for step, (inputs, labels) in enumerate(train_loader):
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.cuda.amp.autocast(enabled=CFG.apex):
            y_preds = model(inputs)
        loss = criterion(y_preds.view(-1, 1), labels.view(-1, 1))
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        scaler.scale(loss).backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            global_step += 1
            if CFG.batch_scheduler:
                scheduler.step()
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(train_loader)-1):
            print('Epoch: [{0}][{1}/{2}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  'Grad: {grad_norm:.4f}  '
                  'LR: {lr:.8f}  '
                  .format(epoch+1, step, len(train_loader),
                          remain=timeSince(start, float(step+1)/len(train_loader)),
                          loss=losses,
                          grad_norm=grad_norm,
                          lr=scheduler.get_lr()[0]))

    return losses.avg


def valid_fn(valid_loader, model, criterion, device):
    losses = AverageMeter()
    model.eval()
    preds = []
    start = end = time.time()
    for step, (inputs, labels) in enumerate(valid_loader):
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        with torch.no_grad():
            y_preds = model(inputs)
        loss = criterion(y_preds.view(-1, 1), labels.view(-1, 1))
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        losses.update(loss.item(), batch_size)
        preds.append(y_preds.sigmoid().to('cpu').numpy())
        end = time.time()
        if step % CFG.print_freq == 0 or step == (len(valid_loader)-1):
            print('EVAL: [{0}/{1}] '
                  'Elapsed {remain:s} '
                  'Loss: {loss.val:.4f}({loss.avg:.4f}) '
                  .format(step, len(valid_loader),
                          loss=losses,
                          remain=timeSince(start, float(step+1)/len(valid_loader))))
    predictions = np.concatenate(preds)
    return losses.avg, np.concatenate(predictions)


def train_loop(folds, fold):

    LOGGER.info(f"========== fold: {fold} training ==========")

    # ====================================================
    # loader
    # ====================================================
    train_folds = folds[folds['fold'] != fold].reset_index(drop=True)
    valid_folds = folds[folds['fold'] == fold].reset_index(drop=True)
    valid_labels = valid_folds['score'].values

    train_dataset = TrainDataset(CFG, train_folds)
    valid_dataset = TrainDataset(CFG, valid_folds)

    train_loader = DataLoader(train_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=True,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=True)
    valid_loader = DataLoader(valid_dataset,
                              batch_size=CFG.batch_size,
                              shuffle=False,
                              num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

    # ====================================================
    # model & optimizer
    # ====================================================
    model = CustomModel(CFG, config_path=None, pretrained=True)
    torch.save(model.config, OUTPUT_DIR + 'config.pth')
    model.to(device)

    def get_optimizer_params(model, encoder_lr, decoder_lr, weight_decay=0.0):
        param_optimizer = list(model.named_parameters())
        no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
        optimizer_parameters = [
            {'params': [p for n, p in model.model.named_parameters() if not any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': weight_decay},
            {'params': [p for n, p in model.model.named_parameters() if any(nd in n for nd in no_decay)],
             'lr': encoder_lr, 'weight_decay': 0.0},
            {'params': [p for n, p in model.named_parameters() if "model" not in n],
             'lr': decoder_lr, 'weight_decay': 0.0}
        ]
        return optimizer_parameters

    optimizer_parameters = get_optimizer_params(model,
                                                encoder_lr=CFG.encoder_lr,
                                                decoder_lr=CFG.decoder_lr,
                                                weight_decay=CFG.weight_decay)
    optimizer = AdamW(optimizer_parameters, lr=CFG.encoder_lr, eps=CFG.eps, betas=CFG.betas)

    # ====================================================
    # scheduler
    # ====================================================
    def get_scheduler(cfg, optimizer, num_train_steps):
        if cfg.scheduler == 'linear':
            scheduler = get_linear_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps
            )
        elif cfg.scheduler == 'cosine':
            scheduler = get_cosine_schedule_with_warmup(
                optimizer, num_warmup_steps=cfg.num_warmup_steps, num_training_steps=num_train_steps, num_cycles=cfg.num_cycles
            )
        return scheduler

    num_train_steps = int(len(train_folds) / CFG.batch_size * CFG.epochs)
    scheduler = get_scheduler(CFG, optimizer, num_train_steps)

    # ====================================================
    # loop
    # ====================================================
    criterion = nn.BCEWithLogitsLoss(reduction="mean")

    best_score = 0.

    for epoch in range(CFG.epochs):

        start_time = time.time()

        # train
        avg_loss = train_fn(fold, train_loader, model, criterion, optimizer, epoch, scheduler, device)

        # eval
        avg_val_loss, predictions = valid_fn(valid_loader, model, criterion, device)

        # scoring
        score = get_score(valid_labels, predictions)

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.4f}  avg_val_loss: {avg_val_loss:.4f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {score:.4f}')

        if best_score < score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')
            torch.save({'model': model.state_dict(),
                        'predictions': predictions},
                        OUTPUT_DIR+f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth")

    predictions = torch.load(OUTPUT_DIR + f"{CFG.model.replace('/', '-')}_fold{fold}_best.pth",
                             map_location=torch.device('cpu'))['predictions']
    valid_folds['pred'] = predictions

    torch.cuda.empty_cache()
    gc.collect()

    return valid_folds

In [ ]:
def get_result(oof_df):
    labels = oof_df['score'].values
    preds = oof_df['pred'].values
    score = get_score(labels, preds)
    LOGGER.info(f'Score: {score:<.4f}')

if CFG.train:
    oof_df = pd.DataFrame()
    for fold in range(CFG.n_fold):
        if fold in CFG.trn_fold:
            _oof_df = train_loop(train, fold)
            oof_df = pd.concat([oof_df, _oof_df])
            LOGGER.info(f"========== fold: {fold} result ==========")
            get_result(_oof_df)
    oof_df = oof_df.reset_index(drop=True)
    LOGGER.info(f"========== CV ==========")
    get_result(oof_df)
    oof_df.to_pickle(OUTPUT_DIR + 'oof_df.pkl')

========== fold: 0 training ==========
/opt/conda/lib/python3.7/site-packages/torch/cuda/amp/autocast_mode.py:114: UserWarning: torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.autocast only affects CUDA ops, but CUDA is not available.  Disabling.")
